In [ ]:
import requests
from io import BytesIO
import imageio
import rasterio
from rasterio.io import MemoryFile
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import json
from requests.auth import HTTPBasicAuth

In [ ]:
class GeoData:
    HOST = 'http://localhost:8000'
    def __init__(self, user, password):
        self.auth=HTTPBasicAuth(user, password)

    @staticmethod
    def _load_image(content):
        return imageio.imread(BytesIO(content))

    @staticmethod
    def _load_raster(content):
        memfile = MemoryFile(content)
        dataset = memfile.open()
        return dataset

    def get_subsampled_raster(self, pk):
        a = requests.get(f'{GeoData.HOST}/api/geoprocess/imagery/subsample/{pk}', auth=self.auth)
        b = requests.get(a.json()['data']['file'], auth=self.auth)
        #return GeoData._load_image(b.content)
        return GeoData._load_raster(b.content)

    def get_image_entry(self, pk):
        a = requests.get(f'{GeoData.HOST}/api/geodata/imagery/{pk}/data', auth=self.auth)
        #return GeoData._load_image(a.content)
        return GeoData._load_raster(a.content)
    
    def post_subsample(self, payload):
        a = requests.post(f'{GeoData.HOST}/api/geoprocess/imagery/subsample', json=payload, auth=self.auth)
        return json.loads(a.content)
    
    def search(self, payload):
        a = requests.get(f'{GeoData.HOST}/api/geosearch', params=payload, auth=self.auth)
        return json.loads(a.content)
    
    def get(self, url):
        a = requests.get(url, auth=self.auth)
        return json.loads(a.content)
        
client = GeoData('user', 'password')

## Search

In [ ]:
bbox = {"type":"Polygon",
 "coordinates":[[[-107.24629793033489,38.821513321590224],
                 [-107.24629793033489,39.21393900330613],
                 [-106.49448944065813,39.21393900330613],
                 [-106.49448944065813,38.821513321590224],
                 [-107.24629793033489,38.821513321590224]]]
}

search_params = {
    "q": json.dumps(bbox),
    "predicate": "within",
    "datatype": "raster",
}

r = client.search(search_params)
r

In [ ]:
sp = r['results'][0]
sp

In [ ]:
raster = client.get(sp['detail'])
raster

## DEM Subsampling

In [ ]:
img_id = raster['image_set']['images'][0]['id']

In [ ]:
src = client.get_image_entry(img_id)
show(src)

In [ ]:
points = np.array([[-107.08212524738178, 39.01040379702808], 
                   [-106.96182164246767, 39.03110215679572], 
                   [-106.90895466037738, 38.98387516880551], 
                   [-106.9805540376965, 38.91038429753703], 
                   [-107.07130208569401, 38.952157178475225], 
                   [-107.08212524738178, 39.01040379702808]])

shape = {"type": "Polygon", 
         "coordinates": [points.tolist(), ]}

In [ ]:
ax = plt.gca()
show(src, ax=ax)
plt.plot(points[:,0], points[:,1], linewidth=5, color='k')

In [ ]:
payload = {
            'source_image': img_id,
            'sample_type': 'geojson',
            'sample_parameters': shape,
        }

result = client.post_subsample(payload)
result

In [ ]:
product = client.get_subsampled_raster(result['id'])

In [ ]:
ax = plt.gca()
show(src, ax=ax, cmap='Greys')
show(product, ax=ax)
plt.plot(points[:,0], points[:,1], linewidth=5, color='k')
plt.show()

## Annotation Subsampling

In [ ]:
img_id = 234

In [ ]:
src = client.get_image_entry(img_id)
# show(src)

In [ ]:
payload = {
            'source_image': img_id,
            'sample_type': 'annotation',
            'sample_parameters': {'id': 251, 'outline': True},
        }

result = client.post_subsample(payload)
result

In [ ]:
product = client.get_subsampled_raster(result['id'])

In [ ]:
ax = plt.gca()
show(src, ax=ax)
show(product, ax=ax, )
ax.set_xlim(0, src.shape[1])
ax.set_ylim(src.shape[0], 0)
plt.show()

In [ ]:
src.shape